In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pymeshfix
import skimage
from joblib import Parallel, delayed
from matplotlib.colors import LightSource
from skimage.io import imread, imsave
from skimage.measure import marching_cubes
from skimage.transform import rescale
from tqdm import tqdm
from trimesh import Trimesh
from trimesh.smoothing import filter_taubin

In [ ]:
import matplotlib

matplotlib.__version__

In [ ]:
def create_surface_plot(time_point):
    mask_dir = f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/lumen_masks_v19_05_2023/"
    if not os.path.exists(
        mask_dir + f"{time_point:04}" + "_lumen_organoid_mask_processed.tif"
    ):
        print("not avalible")
    else:
        combined_masks = imread(
            mask_dir + f"{time_point:04}" + "_lumen_organoid_mask_processed.tif"
        ).copy()
        combined_masks = np.rot90(combined_masks, 3, axes=(1, 2))

        organoid_mask = combined_masks >= 2
        lumen_mask = skimage.morphology.remove_small_objects(
            (combined_masks == 3), min_size=20000 / (0.347 * 0.347 * 2 * 4 * 4)
        ).astype(bool)
        lumen_mask_label = skimage.measure.label(lumen_mask)
        lumen_mask_label = rescale(
            lumen_mask_label,
            [2 / (4 * 0.347), 1, 1],
            order=0,
            anti_aliasing=False,
            preserve_range=True,
        ).astype(np.uint16)
        organoid_mask = rescale(
            organoid_mask,
            [2 / (4 * 0.347), 1, 1],
            order=0,
            anti_aliasing=False,
            preserve_range=True,
        ).astype(np.uint16)

        # Create surface mask
        fig = plt.figure(figsize=(6, 6))
        fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0)
        ax = fig.add_subplot(111, projection="3d")
        ax.set_box_aspect([1, 1, 1])
        # ax.set_box_aspect([1,1,1])
        max_size = (np.array(lumen_mask_label.shape) * 4 * 0.347).max() + 0
        ax.set_xlim3d([0, max_size])
        ax.set_ylim3d([0, max_size])
        ax.set_zlim3d([0, max_size])
        shapes = lumen_mask_label.shape
        # ax.set_box_aspect((np.ptp([0,shapes[1]*4*0.347]), np.ptp([0,shapes[2]*4*0.347]), np.ptp([0,shapes[2]*4*0.347])))
        ax.view_init(15, 15)
        ax.tick_params(axis="both", pad=5)
        ls = LightSource(azdeg=0, altdeg=-65)

        for label in range(1, len(np.unique(lumen_mask_label))):
            # print(label)
            vertices, faces, _, _ = marching_cubes(
                (lumen_mask_label == label).astype(int), 0, step_size=1
            )
            vertices_clean, faces_clean = pymeshfix.clean_from_arrays(vertices, faces)
            vertices_clean = np.round(vertices_clean * 4 * 0.347).astype(int)

            cell_mesh = Trimesh(vertices=vertices_clean, faces=faces_clean)
            cell_mesh = filter_taubin(cell_mesh, iterations=50)

            # Create trisurf plot
            ax.plot_trisurf(
                cell_mesh.vertices[:, 0],
                cell_mesh.vertices[:, 1],
                triangles=cell_mesh.faces,
                Z=cell_mesh.vertices[:, 2],
                lightsource=ls,
                alpha=1.0,
                color="#28d1ce",
            )
        # plt.axis('off')
        vertices, faces, _, _ = marching_cubes(
            (organoid_mask == 1).astype(int), 0, step_size=2
        )
        vertices_clean, faces_clean = pymeshfix.clean_from_arrays(vertices, faces)
        vertices_clean = np.round(vertices_clean * 4 * 0.347).astype(int)

        cell_mesh = Trimesh(vertices=vertices_clean, faces=faces_clean)
        cell_mesh = filter_taubin(cell_mesh, iterations=50)

        # Create trisurf plot
        ax.plot_trisurf(
            cell_mesh.vertices[:, 0],
            cell_mesh.vertices[:, 1],
            triangles=cell_mesh.faces,
            Z=cell_mesh.vertices[:, 2],
            lightsource=ls,
            alpha=0.3,
            cmap="gray",
        )

        ax.xaxis._axinfo["grid"]["color"] = (1, 1, 1, 0)
        ax.yaxis._axinfo["grid"]["color"] = (1, 1, 1, 0)
        ax.zaxis._axinfo["grid"]["color"] = (1, 1, 1, 0)
        xs = [0, 200, 400, 600]
        ax.set_xticks(xs)

        ax.set_xticklabels(ax.get_xticks(), va="bottom")

        # ax.set_zlim(0, max(mask.shape))
        # bbox = fig.bbox_inches.from_bounds(1, 1, 8, 8)
        plt.savefig(
            movie_folder + f"/3D_plot_lumen_{time_point}_{position}.png",
            pad_inches=0,
            dpi=300,
        )
        plt.close()
        # plt.show()

In [ ]:
channels = ["GFP", "mCherry"]
# positions=[10,11,12,13,14,15,16]
positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
# positions=[15,16]
# positions=[6]
times = np.linspace(1, 249, 125, dtype=int)
movie_folder = f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/3D_lumen_movies_v12_07_23/"
if not os.path.exists(movie_folder):
    os.makedirs(movie_folder)


for position in tqdm(positions):
    result = Parallel(n_jobs=31, verbose=7)(
        delayed(create_surface_plot)(i) for i in times
    )

In [ ]:
times = np.linspace(1, 249, 125, dtype=int)
position = 2
time_point = 100

In [ ]:
position == 2 and time_point > times[50:][0]

In [ ]:
movie_folder = f"/cluster/project/treutlein/DATA/imaging/viventis/20210503_201032_6_lines_mosaic_HB4_D4_processed/3D_lumen_movies_v12_07_23/"
positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
times = np.linspace(1, 249, 125, dtype=int)
# Save in extended_figure folder

all_images_3d_lum = []
for time_point in tqdm(times):
    all_image_3d_lum = []
    for position in positions:
        if position == 2 and time_point > times[50:][0]:
            all_image_3d_lum.append((np.zeros(image_3d_lum.shape) + 1))
        else:
            if not os.path.exists(
                movie_folder + f"/3D_plot_lumen_{time_point}_{position}.png"
            ):
                all_image_3d_lum.append((np.zeros(image_3d_lum.shape) + 1))
                print(f"not_found {position} {time_point}")
            else:
                image_3d_lum = imread(
                    movie_folder + f"/3D_plot_lumen_{time_point}_{position}.png"
                ).copy()
                image_3d_lum = image_3d_lum[250:-460, 450:-310]
                image_3d_lum = rescale(image_3d_lum, [0.5, 0.5, 1])
                all_image_3d_lum.append(image_3d_lum)

    all_image_3d_lum = np.array(all_image_3d_lum)
    stack_vert = []
    for i in range(0, len(all_image_3d_lum), 4):
        stack_vert.append(np.hstack(all_image_3d_lum[i : (i + 4)]))
    stack_vert = np.array(stack_vert)
    stack_vert = np.vstack(stack_vert)
    all_images_3d_lum.append((stack_vert * 255).astype(np.uint8))
all_images_3d_lum = np.array(all_images_3d_lum)
import imageio

writer = imageio.get_writer(
    movie_folder + f"3D_plot_lumen_multimosaic_blacked_out.mp4", fps=10
)
for im in tqdm(all_images_3d_lum):
    writer.append_data(im)
writer.close()
imsave(
    movie_folder + f"3D_plot_lumen_multimosaic_blacked_out.tiff",
    np.moveaxis(all_images_3d_lum, -1, 1).astype(np.uint8),
    imagej=True,
    metadata={"axes": "TCYX"},
    compression="zlib",
)